![](https://images.indiatvnews.com/crimecrime/orld_s_cities_1993.jpg)




#  This kernel sets out to answer following questions all with scraping and extracting data from internet. (In other words without pre-existing, pretty formated datasets):

#  1. How can the crime rate of a country be measured?
#  2. Which countries have the highest/lowest crime rates? 
# 3.Are there typical characteristics of countries with high/low crime rates? 
#  4. Are there countries that have different types of crimes that are dominant? 
# 5. Are there also country characteristics that predict trends in crime rates or types of crimes?

 <a id="top"></a> <br>
## Notebook  Content
*  [Introduction/  1.How can the crime rate of a country be measured?](#1)
*  [Analysis of USA first](#2)
1. [Crime data (loading and pre-processing is a bit lentghy)](#3)
1. [2. Which countries have the highest/lowest crime rates? ](#4)
1. [4. Are there countries that have different types of crimes that are dominant? ](#5)
1. [3.Are there typical characteristics of countries with high/low crime rates? ](#6)
1. [5. Are there also country characteristics that predict trends in crime rates or types of crimes?](#7)


<a id="1"></a> <br>
# 1. How can the crime rate of a country be measured?


Starting of with the first question we can use the official definiton of FBI which classifies homicide, rape, robbery and assault  as violent crime and the true criminality of a country wont be skeewed by non-relevant crimes (if such thing exists). Ofcourse some other categories can come into the story, but I think these should be the main aspect. Same metrics that apply for the countries of the world apply for US States (the first analysis) BUT when we collected these measuraments for the countries of the world a strange pattern emerged: Sweden, German etc were on top of the list. And to quote myself as I said (underneath when second question was answered)

"WHAT? It does not make any sence. Sweden? It is because people have more trust, less corruption, better reporting system punishments etc... IN ORDER To mitigate this effect and get the TRUE levels we ought to normalise these values with all of the values/data that I just named, and even more since these are just some of the factors that influence the odd results. That implies finding ALL of these datasets that coincide with ALL of the countries on ALL of the years. "

So we searched for some indicators that could remove this effect and we found "Rule of law" and "Corruption index" (its underneath when reading in the crime data for the entire world). And we did find that this approach paints a better picture of a crime rate of  a country. (As we shall see later on)


In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warnings (as always)


import plotly.figure_factory as ff
import plotly.offline as py
##for online plotting use import plotly.plotly as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from plotly import tools
import gc
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score

First of all we are going to do some **introductory analysis** that should serve as an baby example for the actual one. **Crime Analysis of US States** (with some small Analysis of NYC). Datasets for USA is usually excellant and we did not have to wrestle with it just find a way to answer the questions. If one whishes to skip analysis of USA than scroll down over to 
### Crime Data

<a id="2"></a> <br>
# US States Analysis


Now let us load an US data set containing 4 types of violent crime in major US cities (cities with over 250 000 citizens). This should serve as a templete to an (almost) perfect data set that we need to find. We have 40 years of data, there are 4 types of different crimes reported all through out the year. Further more we have clearly distinct areas with additional number of population for each area.

In [ ]:
df2 = pd.read_csv("../input/crime-rates/report.csv", encoding='latin1')
df2

Now us do some EDA to get to know our complete data set

In [ ]:
ax=plt.figure(figsize=(10,22))
ax = sns.barplot(df2["violent_crimes"]+df2["homicides"]+
                 df2["rapes"]+df2["assaults"]+df2["robberies"],                 
                 y=df2["agency_jurisdiction"],estimator=sum,color="y")
ax.set_title('Total number of crimes in US during in 40 years')
ax.set(xlabel='Total number of crimes', ylabel='City and state')

There are a couple of things that are interesting here. First of all city "United States" is the sum of them all. Further more New York, LA, Chicago seem to be most violent, or is it because of the shere number of citizens in these towns? How can we investigate it? Let us creeate a new variable that is relative to the number of citizens, i.e. total number of crime divided with total citizens. (NOTE: Black line is simply confidence interval of estimation)

# Main metric that we are trying to predict:

In [ ]:

df2["relative_crime"]=(df2["violent_crimes"]+df2["homicides"]+
                 df2["rapes"]+df2["assaults"]+df2["robberies"])/df2.population



In [ ]:
ax=plt.figure(figsize=(10,22))
ax = sns.barplot(x=df2["relative_crime"],y=df2["agency_jurisdiction"],color="b")
ax.set_title('Total number of crimes in US during in 40 years')
ax.set(xlabel='Total number of crimes', ylabel='City and state')

That paints another picture, now top of the list is St.Louis, Atlanta, Miami etc. That was interesting, but was it always like that? What was the number of crimes over the years?

In [ ]:
i = df2[(df2.agency_jurisdiction == "United States")].index
df2.drop(i,inplace=True)


In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

a = set(df2["agency_jurisdiction"])
a = list(a)

doubles = dict()
for i in range(0,len(a)):
    doubles[i] = df2[df2['agency_jurisdiction'].str.contains(a[i])]

trace = dict()
for i in range(0,len(a)):
    trace[i] = go.Scatter(x = doubles[i]['report_year'],y=doubles[i]['violent_crimes'],name = a[i],opacity = 0.8)

data = [trace[0],trace[1],trace[2],trace[3],trace[4],trace[5],trace[6],trace[7],trace[8],trace[9],
        trace[10],trace[11],trace[12],trace[13],trace[14],trace[15],trace[16],trace[17],trace[18],trace[19],
         trace[20],trace[21],trace[22],trace[23],trace[24],trace[25],trace[26],trace[27],trace[28],trace[29],
          trace[30],trace[31],trace[32],trace[33],trace[34],trace[35],trace[36],trace[37],trace[38],trace[39],
           trace[40],trace[41],trace[42],trace[43],trace[44],trace[45],trace[46],trace[47],trace[48],trace[49],
            trace[50],trace[51],trace[52],trace[53],trace[54],trace[55],trace[56],trace[57],trace[58],trace[59],
             trace[60],trace[61],trace[62],trace[63],trace[64],trace[65],trace[66],trace[67]]

layout = dict(title = "Total Crimes in US during in 40 years",
              xaxis = dict(title = 'Time Span'),
              yaxis = dict(title = 'Cumulative crimes'),)

fig = dict(data=data, layout=layout)
py.iplot(fig)

Now from here we can set a lot of interesting question. One of them would be why was crime in NYC in 90-s so high? Well there seems to be a couple of factors (we state them, not test them) For example, the police force in New York City grew by 35 percent in the 1990s, the numbers of prison inmates rose 24 percent, and there were demographic changes, including a decline in the number of youths. The national unemployment rate declined 25 percent between 1990 and 1999, and by 39 percent in the city between 1992 and 1999. This study shows that a single percentage point decline in the jobless rate decreases burglary by 2.2 percent and motor vehicle theft by 1.8 percent. Increases in the real minimum wage also significantly reduce robberies and murders: 3.4 to 3.7 percent fewer robberies with a 10 percent increase in the minimum wage and 6.3 to 6.9 percent fewer murders. And ofcourse police measures.The police measure that most consistently reduces crime is the arrest rate of those involved in crime, the study finds. Felony arrest rates (except for motor vehicle thefts) rose 50 to 70 percent in the 1990s. When arrests of burglars increased 10 percent, the number of burglaries fell 2.7 to 3.2 percent. When the arrest rate of robbers rose 10 percent, the number of robberies fell 5.7 to 5.9 percent. In the case of murder, the decline was 3.9 to 4 percent; in the case of assault, 2 to 2.4 percent; and for motor vehicle theft, 5 to 5.1 percent. Could a mayor have helped in reducing crime?

In [ ]:
NYC = df2[df2.agency_jurisdiction == 'New York City, NY']

ax=plt.figure(5,figsize=(18,12))
ax = plt.plot([1975, 1975], [0, 193000], 'y-', lw=2)
ax = plt.text(1975, 193000, 'Abraham Beame',color='blue',horizontalalignment='center')
ax = plt.plot([1978, 1978], [0, 193000], 'r-', lw=2)
ax = plt.text(1978, 193000, 'Ed Koch',color='blue',horizontalalignment='left')
ax = plt.plot([1990, 1990], [0, 193000], 'g-', lw=2)
ax = plt.text(1990, 193000, 'David Dinkins',color='blue',horizontalalignment='center')
ax = plt.plot([1994, 1994], [0, 193000], 'y-', lw=2)
ax = plt.text(1994, 193000, 'Rudy Giuliani',color='red',horizontalalignment='center')
ax = plt.plot([2002, 2002], [0, 193000], 'r-', lw=2)
ax = plt.text(2002, 193000, 'Michael Bloomberg',color='red',horizontalalignment='center')
ax = plt.plot([2014, 2014], [0, 193000], 'g-', lw=2)
ax = plt.text(2014, 193000, 'Bill de Blasio',color='blue',horizontalalignment='center')
ax = plt.plot(NYC["report_year"],NYC["violent_crimes"])
plt.title('Crimes while mayors')
plt.xlabel("Year")
plt.ylabel("Sum of all crimes")
plt.ylim([0,220000])
plt.xlim([1970,2020])
plt.show()

Let us now jumpt to conculusions, correlation is not causation but David Dinkins must have helped. New York city seems interesting since on the absolute scale it has the most crime in the entire United States. Let us dig deeper in NYC.

In [ ]:

ax=plt.figure(figsize=(15,10))
plt.xticks(rotation='vertical')
ax=sns.barplot(NYC["report_year"], NYC["homicides"],palette="cubehelix")

plt.ylabel("Number of Homicides")

In [ ]:
ax=plt.figure(figsize=(15,10))
plt.xticks(rotation='vertical')
sns.barplot(NYC["report_year"], NYC["rapes"],palette="cubehelix")
plt.ylabel("Number of Rapes")

In [ ]:
ax=plt.figure(figsize=(15,10))
plt.xticks(rotation='vertical')
sns.barplot(NYC["report_year"], NYC["assaults"],palette="cubehelix")
plt.ylabel("Number of Assaults")

In [ ]:
ax=plt.figure(figsize=(15,10))
plt.xticks(rotation='vertical')
sns.barplot(NYC["report_year"], NYC["robberies"],palette="cubehelix")
plt.ylabel("Number of Robberies")

We have analyzed NYC enough, let us now try to find some socio-economic data, and try to see if we can use it to predict crime rate all around US. But how do we quantify successful state? NY state is quite successful at the first glance, but it is at the top of our list for crime. A couple that we should look for and that are at the top of my mind are: Unemployment Rate,Worker Productivity,Startup Activity,Median Household Income,Upward Mobility Rates,Road Quality,Public Utility Fee Rates,Overall Tax Burden  etc etc... I managed to dig up 2 interesting datasets. First if US unemplyment by county and state from 1990 until 2016 and Zillow Rent Index from 2010 until 2017. Hypothesis is simple, where there was a lot of unemplyment and biggest living factor (rent) was high ->>> more crime. Let us test that. But before we do that, we need to merge the datasets together.

In [ ]:
df3 = pd.read_csv("../input/unemployment-by-county-us/output.csv", encoding='latin1')
df3.head()
df3['County']=df3['County'].astype(str).str.split().str[0]
df3['County']

In [ ]:
df3.rename(columns={'Rate':'Unemployment rate'}, inplace=True)

In [ ]:
df3.head()

WE have from 2009 until 2015 unemployment rates.

In [ ]:
df3.isna().sum().sum()

In [ ]:
df4 = pd.read_csv("../input/rent-index/price.csv", encoding='latin1')
df4.head()

In [ ]:
df4.describe()

In [ ]:
years = list(set([y.split( )[1] for y in df4.columns[6:]]))
months = df4.columns[6:]

Lets have a look at rent at New Jersey and NYC, monthly! Before that we had to split the months from the dataframe

In [ ]:
nyc_rent = df4[df4["Metro"]=="New York"]
ny_njer_rent = df4.groupby("State")[months].median()


In [ ]:
trace1 = go.Scatter(x = months, 
                  y= np.nanmedian(nyc_rent[nyc_rent["State"]=="NY"][months], axis = 0),
                    mode='markers', marker=dict(size=5,color = ('aqua')), 
                   name = "New York")
trace2 = go.Scatter(x = months, 
                  y= np.nanmedian(nyc_rent[nyc_rent["State"]=="NJ"][months], axis = 0),
                    mode='markers', marker=dict(size=5,color = ('navy')),
                   name = "New Jersey")



fig = tools.make_subplots(rows= 1 , cols=2, subplot_titles=('Median NY Rent','Median NJ Rent'))
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 1,2)

fig['layout'].update(showlegend=False, title='Median Rent Price of New York VS New Jersey')
py.iplot(fig)

In [ ]:
df4.isna().sum().sum()

We have to analyse and deal with there missing values. First of all let us consult dataset description, maybe there something there to be found. According to the documentation time series goes back to november 2010 but nor for every city/county/state. (Obviously) So we are left with no hints on how to handle the NaN. MICE/ using other columns as indicators to imputate missing values makes really no sense since these are rising time series, on monthly basis.  Sinc

Only rows that have missing values:

In [ ]:
df4[df4.isnull().any(axis=1)].head()

But where are these missing values mostly missing?

In [ ]:
sns.heatmap(df4.isnull(), cbar=False)

It seems that all of the missing values are mostly just in the first year. Interpolation is hard because we need values from both sides, and NaN start right from the start. WE could imputate with mean but we reduce the volatility of the data set. Another thing to do here is to cut the first year (so we are starting with November 2011) and for the rest just use backfill. This approach is bad, since we are only throwing less then 1/6 (still a considerable amount of data!).
### Solution: Just use linear regression since the values do follow it

In [ ]:
df4.interpolate(method='piecewise_polynomial', inplace=True, limit_direction="both")

In [ ]:
df4

In [ ]:
df4.isna().sum().sum()

It seems that we missed a spot, it is because of the piecewise polynomial method. No problem, for the rest 1245 Values (not rows!) we can use linear regression, since (looking at the graphs polynomials seem more reasonable but linear regression is not far from truth!)

In [ ]:
df4.interpolate(method='linear', inplace=True, limit_direction="both")

In [ ]:
df4.drop(["Metro"], axis=1,inplace=True)

In [ ]:
df4.isna().sum().sum()

**df2-Data frame**, dataset of crimes for the entire US, including the desired column "relative_crime" also has some NaN, let us inspect whats wrong

In [ ]:
df2.isna().sum().sum()

In [ ]:
sns.heatmap(df2.isnull(), cbar=False)

How to interpret this heatmap? y-axis are indices (in this case nubmers) Because of the shere number of them we can only approximately find them, but atleast we see what are the troubling columns.

In [ ]:
df2=df2[df2["report_year"]>2009]

In [ ]:
df2.drop(["months_reported","agency_code"], axis=1,inplace=True)

In [ ]:
df2[df2.isnull().any(axis=1)]

Now here we can clearly see that the missing values are (mostly!) just missing yearly measures. Since we already know that we only have 2010 values for rent across the whole USA, than thats what we are going to be looking at here, hence only 74 rows with (atleast one) missing value. Effectively we only have 3 rows that have missing values. Let us just drop them.

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df2.isna().sum().sum()

Now that we cleaned our datasets we can merge them. But before that we ought to split the "agency_jurisdiction" into two columns containing counties.

In [ ]:
df2.head()

In [ ]:
df2[['City',"State"]] = df2["agency_jurisdiction"].str.split(',',expand=True)

In [ ]:
df2.drop(["City","agency_jurisdiction"], axis=1,inplace=True)

In [ ]:
df2.head()

df3 is a bit problematic, that is why we are going to need to create another dataframe that is going to help us to have abbreviations for states and not the full names.

In [ ]:
d = {'State': ["Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois", "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania", "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"], 'Abb': ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]}
df_abv = pd.DataFrame(data=d)

In [ ]:
df_abv.head()

In [ ]:
df3=df3.merge(df_abv)
df3.drop(["State"], axis=1,inplace=True)

In [ ]:
df3.rename(columns={'Abb':'State'}, inplace=True)

In [ ]:
df3.head()

In [ ]:
df = df3.merge(df4)
df.drop(["Month","County","City Code","Population Rank","City"], axis=1,inplace=True)
df=df[df["Year"]>2009]


Before merging with df2, the crime dataset, we have to think about memory of the df dataset. There are a lot of rows that we basically wont need. hence we can drop them

In [ ]:
df.State = df.State.astype(str)
df2.State = df2.State.astype(str)
df.State=df.State.str.strip()
df2.State=df2.State.str.strip()


For memory issues we will have to make this aggregation, in order to save space. Alternative is this kernel https://www.kaggle.com/o7ronak7/how-to-save-time-and-memory-with-big-datasets

In [ ]:
df=df.groupby(["State","Year"]).mean()

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

In [ ]:
final_df=df2.merge(df)


In [ ]:
del df2,df3,df4,df
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
X_train_set_df=final_df[final_df["Year"]<2015].drop(["State","report_year","Year"], axis=1)
Y_train_set_df=final_df[final_df["Year"]<2015].drop(['report_year', 'population', 'violent_crimes', 'homicides', 'rapes',
       'assaults', 'robberies', 'crimes_percapita', 'homicides_percapita',
       'rapes_percapita', 'assaults_percapita', 'robberies_percapita',
       'State', 'Year', 'Unemployment rate', 'November 2010',
       'December 2010', 'January 2011', 'February 2011', 'March 2011',
       'April 2011', 'May 2011', 'June 2011', 'July 2011', 'August 2011',
       'September 2011', 'October 2011', 'November 2011', 'December 2011',
       'January 2012', 'February 2012', 'March 2012', 'April 2012', 'May 2012',
       'June 2012', 'July 2012', 'August 2012', 'September 2012',
       'October 2012', 'November 2012', 'December 2012', 'January 2013',
       'February 2013', 'March 2013', 'April 2013', 'May 2013', 'June 2013',
       'July 2013', 'August 2013', 'September 2013', 'October 2013',
       'November 2013', 'December 2013', 'January 2014', 'February 2014',
       'March 2014', 'April 2014', 'May 2014', 'June 2014', 'July 2014',
       'August 2014', 'September 2014', 'October 2014', 'November 2014',
       'December 2014', 'January 2015', 'February 2015', 'March 2015',
       'April 2015', 'May 2015', 'June 2015', 'July 2015', 'August 2015',
       'September 2015', 'October 2015', 'November 2015', 'December 2015',
       'January 2016', 'February 2016', 'March 2016', 'April 2016', 'May 2016',
       'June 2016', 'July 2016', 'August 2016', 'September 2016',
       'October 2016', 'November 2016', 'December 2016', 'January 2017'], axis=1)

In [ ]:
X_test_set_df=final_df[final_df["Year"]>2014].drop(["State","report_year","Year"], axis=1)
Y_test_set_df=final_df[final_df["Year"]>2014].drop(['report_year', 'population', 'violent_crimes', 'homicides', 'rapes',
       'assaults', 'robberies', 'crimes_percapita', 'homicides_percapita',
       'rapes_percapita', 'assaults_percapita', 'robberies_percapita',
       'State', 'Year', 'Unemployment rate', 'November 2010',
       'December 2010', 'January 2011', 'February 2011', 'March 2011',
       'April 2011', 'May 2011', 'June 2011', 'July 2011', 'August 2011',
       'September 2011', 'October 2011', 'November 2011', 'December 2011',
       'January 2012', 'February 2012', 'March 2012', 'April 2012', 'May 2012',
       'June 2012', 'July 2012', 'August 2012', 'September 2012',
       'October 2012', 'November 2012', 'December 2012', 'January 2013',
       'February 2013', 'March 2013', 'April 2013', 'May 2013', 'June 2013',
       'July 2013', 'August 2013', 'September 2013', 'October 2013',
       'November 2013', 'December 2013', 'January 2014', 'February 2014',
       'March 2014', 'April 2014', 'May 2014', 'June 2014', 'July 2014',
       'August 2014', 'September 2014', 'October 2014', 'November 2014',
       'December 2014', 'January 2015', 'February 2015', 'March 2015',
       'April 2015', 'May 2015', 'June 2015', 'July 2015', 'August 2015',
       'September 2015', 'October 2015', 'November 2015', 'December 2015',
       'January 2016', 'February 2016', 'March 2016', 'April 2016', 'May 2016',
       'June 2016', 'July 2016', 'August 2016', 'September 2016',
       'October 2016', 'November 2016', 'December 2016', 'January 2017'], axis=1)

Modelling: Elastic Net (in essence regression with penalization term) regression

In [ ]:
# Prints R2 and RMSE scores
def get_score(prediction, lables):    
    print('R2: {}'.format(r2_score(prediction, lables)))
    print('RMSE: {}'.format(np.sqrt(mean_squared_error(prediction, lables))))

# Shows scores for train and validation sets    
def train_test(estimator, x_trn, x_tst, y_trn, y_tst):
    prediction_train = estimator.predict(x_trn)
    # Printing estimator
    print(estimator)
    # Printing train scores
    get_score(prediction_train, y_trn)
    prediction_test = estimator.predict(x_tst)
    # Printing test scores
    print("Test")
    get_score(prediction_test, y_tst)

In [ ]:
ENSTest = linear_model.ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.01, 0.1, 1, 10], l1_ratio=[.01, .1, .5, .9, .99], max_iter=5000).fit(X_train_set_df, Y_train_set_df)
train_test(ENSTest, X_train_set_df, X_test_set_df, Y_train_set_df, Y_test_set_df)

That was a small taste of what an analysis should look like on a US states. For a larger scale analysis first we have to gather/scrape data from verious sources.

We need to find datasets of Crime, Socio and Economic factors of different countries expanding over a large time horizont.

<a id="3"></a> <br>
# Crime Data

In [ ]:
df = pd.read_csv('../input/crimes-un-datacsv/Crimes_UN_data.csv', skiprows=[0], encoding='latin1')
df.head()

Very ugly data frame, let us try to clean it up and keep only the desired columns

In [ ]:
cols = list(df.columns)
cols[1] = 'Region'
df.columns = cols
pivoted = pd.pivot_table(df, values='Value', index=['Region', 'Year'], columns='Series', aggfunc=sum)
un_crimes=pivoted
pivoted.head()

How do we even impute this? Basic methods like foreward fill and others are out of the question since they are obviously false for many reasons. There is an option to import another data set containing socio-economic variables and based on that try to predict the missing values. In essence build a model to predict the missing values. Problem with that is that we have too little training data, i.e. for Afganistan we have only two. Algorithm can not learn with that. I decided to set the NaN values to zero and atleast extract some statistics and interesing plots from the data at hand. And we are going to supplement with other data sources later on.

In [ ]:
pivoted=pivoted.fillna(0)

Second question can partially be answered with the dataset at hand:

1. Which countries have the highest/lowest crime rates?(Full/complementary answer later on)

3. Obviously very rialable signs of a country crime rate are 5 columns : Assault rate per 100,000 population, Intentional homicide rates per 100,000, Robbery at the national level, rate per 100,000 population, Theft at the national level, rate per 100,000 population, Total Sexual Violence at the national level, rate per 100,000 They are all normed to 100 000 citizens and describe various hard-crimes.

4. We can answer with Table of maximum values for all 5 different types of crime

In [ ]:
pivoted.loc[pivoted.index.get_level_values(0)]

In [ ]:
pivoted_max=pivoted.max(level='Region')
pivoted_max=pivoted_max.fillna(0)
pivoted_max["Intentional homicide rates per 100,000"]=pivoted_max["Intentional homicide rates per 100,000"].astype('float')
pivoted_max["Robbery at the national level, rate per 100,000 population"]=pivoted_max["Robbery at the national level, rate per 100,000 population"].astype('float')
pivoted_max["Assault rate per 100,000 population"]=pivoted_max["Assault rate per 100,000 population"].astype('float')

pivoted_max1=pivoted_max.sort_values(by="Intentional homicide rates per 100,000",ascending=False)
pivoted_max

It appears that the second column "Intentional homicide rates per 100,000" is mostly populated, so we can make some inference from it!

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 10))
pivoted_max1['Intentional homicide rates per 100,000'].head(15).plot(kind='bar', rot=90,  title='Intentional homicide rates per 100,000')

In [ ]:
pivoted_max2=pivoted_max.sort_values(by="Robbery at the national level, rate per 100,000 population",ascending=False)
fig, ax = plt.subplots(figsize=(20, 10))
pivoted_max2['Robbery at the national level, rate per 100,000 population'].head(15).plot(kind='bar', rot=90,  title='Robbery at the national level, rate per 100,000 population')

In [ ]:
pivoted_max3=pivoted_max.sort_values(by="Assault rate per 100,000 population",ascending=False)
fig, ax = plt.subplots(figsize=(20, 10))
pivoted_max3["Assault rate per 100,000 population"].head(15).plot(kind='bar', rot=90,  title='Assault rate per 100,000 population')

In [ ]:
del pivoted_max3,pivoted_max,df,pivoted_max2,pivoted


gc.collect()

Some interesting plots (albeit not complete data! only introductory!) but it seems that there is a suprising prevelance of assault rate in unexpected places(as already stated in the beginning). Sweden, Belgium, Germany certainly do not classify as dangerous places to live. Assumption is that these countries have better/more effective police force that is able to cover and handle these types of crimes more effectively hence there are more assaults. If we were to compare that with worse crimes like murder, than the police can not look the other way and it is well documented that usuall countries will the top of the list.

# Nationmaster

Starting of with finding the crime data for the whole world, for a big number of years:

In [ ]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20


In [ ]:
nm = pd.read_csv('../input/nationmaster/Nationmaster.csv', encoding='latin1')
nm.rename(columns={'Unnamed: 0': "Country"}, inplace=1)
nm.shape


In [ ]:
#dropping data that is irrelevant and 
nm.drop(columns=[
	"Age of criminal responsibility %28notes%29", "Background",
	"Illicit drugs",
	"Minimum to serve before eligibility for requesting parole",
	"Prosecution rate",
	"Prosecution rate per million",
	"%25 sales"
], inplace=True)

print("deleting average countries: \n", nm[nm["Country"].str.match(".*average")]["Country"])
nm.drop(nm[nm["Country"].str.match(".*average")].index, inplace=True)


In [ ]:
#dropping countries we do not have data about
limit = 20  # drop all countries with data in less than x columns
bad_countries = (~nm.isna()).sum(axis=1)
print("deleting ", (bad_countries < 20).sum())
nm.drop(nm[bad_countries < 20].index, inplace=True)
nm.shape


In [ ]:
nm[nm["Country"].str.match(".*Un.*")]  # country name search


In [ ]:
nm2 = nm.copy()


In [ ]:
#dropping columns we do not have rows for enough countries
nm = nm2.copy()
limit = 60  # drop columns with more than x countries missing
bad_cols = nm.isna().sum(axis=0)
print("deleting ", (bad_cols > limit).sum(), ":\n", nm.columns[bad_cols > limit])
nm.drop(columns=nm.columns[bad_cols > limit],inplace=True)
print(nm.shape)
nm

In [ ]:
import plotly.offline as py

import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [ ]:
countriesraw = pd.read_csv('../input/plotly_countries/2014_world_gdp_with_codes.csv')
countries=countriesraw.copy()

countries["Country"]=countries["COUNTRY"]
countries=countries.drop(columns=["COUNTRY"])

nm.reset_index(inplace=True)

nm=nm.merge(countries,how="outer")

nm.sort_index(inplace=True)
nm.set_index("Country",inplace=True)
nm.columns

In [ ]:
column_name='Total crimes per 1000'
data = [ dict(
        type = 'choropleth',
        locations = nm['CODE'],
        z = nm[column_name].fillna(0),
        text = nm.index,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = column_name),
      ) ]

layout = dict(
    title = 'world plot of '+column_name,
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

nm[[column_name]].sort_values(by=column_name,ascending=0)


# Importing UNO DC Crime data

In [ ]:
unodc=None
fields=["Assault","Kidnapping","Theft","Robbery","Burglary","Domestic_Burglary","Theft_of_Private_Cars","Motor_Vehicle_Theft",
        "Total_Sexual_Violence","Rape","Sexual_Offences_ag_Children"]
string="abcdefghijk"

for i in range(len(string)):
    l=string[i]
    names=["%s_Rel_20%02d"%(fields[i],j) for j in range(3,15)]
    
    tmp= pd.read_csv('../input/unodc_crime/Publication Reports_1%s.csv'%l,sep=";",index_col=[2],header=None,skipinitialspace=True,skiprows=10, encoding='latin1',
                 names=[
        "Region","Sub-Region","Country",
        *["%s_Abs_20%02d"%(fields[i],j) for j in range(3,15)],
                     "nothing",
        *["%s_Rel_20%02d"%(fields[i],j) for j in range(3,15)]  
        ],
        
    )
    tmp.reset_index(inplace=True)
    tmp["Country"]=tmp["Country"].str.replace("*","").str.replace(" of America","").str.replace(" (England and Wales)","",regex=False).str.replace("Russian Federation","Russia").str.replace("Republic of ","").str.replace(" (Plurinational State of)","",regex=False).str.replace("Korea","Korea, South",regex=False)
    tmp.set_index("Country",inplace=True)
    tmp.drop(columns=["Region","Sub-Region","nothing"],inplace=True)
    for n in names:
        tmp[n]=tmp[n].str.replace(".","").str.replace(",([0-9])$",lambda m: "."+m[1]).astype(float)
    tmp=tmp.loc[tmp.index.dropna()]
    
    unodc=tmp  if unodc is None else unodc.merge(tmp,how="outer",on="Country")
unodc.drop("Country/territory",inplace=True)
#unodc.rename(columns={'Country/territory':'Subcontinent'},inplace=True)

In [ ]:
countries=pd.read_csv('../input/plotly_countries/2014_world_gdp_with_codes.csv')
countries["Country"]=countries["COUNTRY"]
countries=countries.drop(columns=["COUNTRY","GDP (BILLIONS)"])


#unodc["CC"]=countries.loc[unodc["Country"].str.replace("*","")].reset_index()["CODE"]
unodc=unodc.reset_index().merge(countries,how="outer")

unodc.sort_index(inplace=True)
unodc.set_index("Country",inplace=True)

In [ ]:
unodc_mult=pd.DataFrame()
tmp=pd.DataFrame()
for i in range(3,15):
    tmp=unodc[["%s_Abs_20%02d"%(f,i) for f in fields]+["%s_Rel_20%02d"%(f,i) for f in fields]]
    tmp.columns=tmp.columns.str.replace("_20%02d"%i,"")
    tmp["Year"]=2000+i
    tmp=tmp.reset_index().set_index(["Country","Year"])
    unodc_mult=unodc_mult.append(tmp)
unodc_mult=unodc_mult.sort_index()
unodc_mult = unodc_mult.reset_index()
unodc_mult.rename(columns={'Country':'Region'},inplace=True)
unodc_mult = unodc_mult.set_index(['Region','Year'])
unodc_mult[:15]

In [ ]:
df=pd.read_csv('../input/crimes-un-datacsv/Crimes_UN_data.csv', skiprows=[0],thousands=",", encoding='latin1')
cols = list(df.columns)
cols[1] = 'Region'
df.columns = cols
pivoted = pd.pivot_table(df, values='Value', index=['Region', 'Year'], columns='Series', aggfunc=sum)

un_crimes=pivoted
un_crimes=un_crimes.rename({"United States of America": "United States"}, axis='index')

In [ ]:
unodc_un=unodc_mult.merge(un_crimes, left_index=True, right_index=True,how="outer")
unodc_un.columns=unodc_un.columns.str.replace(" per 100,000"," Rel",regex=False).str.replace(" population","",regex=False)
unodc_un.index.levels[0][unodc_un.index.levels[0].str.match("United")]

In [ ]:
column=fields[2]
year=3
column_name="%s_Rel_20%02d"%(column,year) 
plot=unodc[column_name].fillna(0)#.str.replace(',', '')
CCs=unodc['CODE']
data = [ dict(
        type = 'choropleth',
        locations = CCs,
        z = plot,
        text = unodc.index,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = column_name.replace("_"," ")),
      ) ]

layout = dict(
    title = 'world plot of '+column_name.replace("_"," "),
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )
#plot.sort_values(ascending=False)[:20]

<a id="4"></a> <br>
### Question 2: Which countries have the highest crime crates?
Approach: take all the different categories of crimes we have data about, average them over the years if existing, then normalising them to 
my=0, sigma=1, and then finding an average of the categories for each country. 

In [ ]:
unodc_avg_year=pd.DataFrame()
unodc_avg=pd.DataFrame()
unodc_max_cat=pd.DataFrame()
for f in fields: # trying to use only these to not let rare crimes play big role, but no difference:
                #["Robbery","Assault","Kidnapping","Burglary","Theft_of_Private_Cars"]:
    names=["%s_Rel_20%02d"%(f,j) for j in range(3,15)]
    unodc_avg_year[f]=unodc[names].mean(axis=1)
un_fields=['Assault rate Rel',
       'Intentional homicide rates Rel',
       'Kidnapping at the national level, rate Rel',
       'Percentage of male and female intentional homicide victims, Female',
       'Percentage of male and female intentional homicide victims, Male',
       'Robbery at the national level, rate Rel',
       'Theft at the national level, rate Rel',
       'Total Sexual Violence at the national level, rate Rel']
for f in un_fields:
    print(f)
    names=["%s_Rel_20%02d"%(f,j) for j in range(3,15)]
    unodc_avg_year[f]=unodc_un[f].unstack(level=1).astype(float).mean(axis=1)
    
unodc_avg_year-=unodc_avg_year.mean()
unodc_avg_year/=unodc_avg_year.std()
unodc_avg["crime_score"]=unodc_avg_year.mean(axis=1)
unodc_avg["crime_score"]-=unodc_avg["crime_score"].min()
unodc_avg["crime_score"].fillna(0,inplace=True)
unodc_avg["CODE"]=unodc["CODE"]
unodc_avg_year.loc[["United States","Sweden","Austria","Mexico","Morocco","Serbia"]]

WHAT? It does not make any sence. Sweden? It is because people have more trust, less corruption, better reporting system punishments etc... IN ORDER To mitigate this effect and get the TRUE levels we ought to normalise these values with all of the values/data that I just named, and even more since these are (probably) correct assumptions of why the results are odd. That implies finding ALL of these datasets that coincide with ALL of the countries on ALL of the years. We are going find some data two rows underneath and than normalise total crime for a better result.

In [ ]:
column_name="crime_score"
data = [ dict(
        type = 'choropleth',
        locations = unodc_avg['CODE'],
        z = unodc_avg['crime_score'],
        text = unodc.index,
        colorscale =[[0.0, 'rgb(255,255,255)'],[1e-6, 'rgb(237,235,242)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']],
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            #autotick = False,
            title = column_name.replace("_"," ")),
      ) ]

layout = dict(
    title = 'world plot of '+column_name.replace("_"," "),
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )
#import plotly.io as pio
#pio.write_image(fig, 'world_avg_cr.pdf')
unodc_avg[["crime_score"]].sort_values(by="crime_score",ascending=False)[:10]

In [ ]:
corruption=pd.read_csv("../input/unodc_crime/theglobaleconomy_corruption.csv", sep=", ")
corruption["Rule of law"]=corruption["Rule of law"].astype(float)

corruption["Corruption perceptions - Transparency International"]=corruption["Corruption perceptions - Transparency International"].str.replace(",","").fillna(-1).astype(int).replace(-1, np.nan)
corruption.set_index(["Country","Code","Year"],inplace=True)
corruption_avg=pd.DataFrame()
corruption
corruption_avg["Rule of law"]=2.5-corruption["Rule of law"].unstack(level=2).mean(axis=1)
corruption_avg["Corruption Perception"]=100-corruption["Corruption perceptions - Transparency International"].unstack(level=2).mean(axis=1)
corruption_avg[["Rule of law","Corruption Perception"]]-=corruption_avg[["Rule of law","Corruption Perception"]].mean(axis=0)
corruption_avg[["Rule of law","Corruption Perception"]]/=corruption_avg[["Rule of law","Corruption Perception"]].std(axis=0)
corruption_avg["corruption_score"]=corruption_avg.mean(axis=1)
corruption_avg.reset_index(inplace=True)
corruption_avg.set_index(["Country"],inplace=True)
corruption_avg[:10]

In [ ]:
column_name="corruption"
data = [ dict(
        type = 'choropleth',
        locations = corruption_avg['Code'],
        z = corruption_avg['corruption_score'],
        text = corruption_avg.index,
        colorscale =[[0.0, 'rgb(255,255,255)'],[1e-6, 'rgb(237,235,242)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']],
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            #autotick = False,
            title = column_name.replace("_"," ")),
      ) ]

layout = dict(
    title = 'world plot of '+column_name.replace("_"," "),
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )
#import plotly.io as pio
#pio.write_image(fig, 'world_avg_cr.pdf')
corruption_avg[["corruption_score"]].sort_values(by="corruption_score",ascending=False)[:10]

In [ ]:
unodc_w=unodc_avg.reset_index().merge(corruption_avg,left_on="CODE",right_on="Code").set_index("Country")
unodc_w["crime_score_w"]=unodc_w["crime_score"]+(unodc_w["corruption_score"]/3)
unodc_w.sort_values(by="crime_score_w",ascending=False)[:10]

In [ ]:
column_name="crime_score_including_corruption"
data = [ dict(
        type = 'choropleth',
        locations = unodc_w['CODE'],
        z = unodc_w['crime_score_w'],
        text = unodc_w.index,
        colorscale =[[0.0, 'rgb(255,255,255)'],[1e-6, 'rgb(237,235,242)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']],
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            #autotick = False,
            title = column_name.replace("_"," ")),
      ) ]

layout = dict(
    title = 'world plot of '+column_name.replace("_"," "),
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )
#import plotly.io as pio
#pio.write_image(fig, 'world_avg_cr.pdf')

<a id="5"></a> <br>
### Question 4: Do different countries have different types of crimes dominating?

In [ ]:
unodc_max_cat["max_cat"]=unodc_avg_year[fields].idxmax(axis=1)
unodc_max_cat["CODE"]=unodc["CODE"]
unodc_max_cat["max_cat_ind"]=unodc_max_cat["max_cat"].map(dict((f[1],f[0]+1)for f in enumerate(fields)))
unodc_max_cat["max_cat_ind"]=unodc_max_cat["max_cat_ind"].fillna(0)

In [ ]:
column_name="world_max_crime_category"
colors=['#ffffff','#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', 
'#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#000000']
data = [ dict(
        type = 'choropleth',
        locations = unodc_max_cat['CODE'],
        z = unodc_max_cat['max_cat_ind'],
        text = unodc_max_cat["max_cat"].str.replace("_"," "),
        colorscale = [[i/11, colors[i]] for i in range(0,12)],
        autocolorscale = False,
        reversescale = False,
        show_scale=True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = True,
            title = "colors"
        ),
      ) ]

layout = dict(
    title = 'world plot of '+column_name.replace("_"," "),
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map2' )
unodc_max_cat[["max_cat"]].loc[["Austria","United States","United Kingdom","Serbia","Croatia","India"]]

# ECONOMIC FACTORS


In this section we are going to load and preprocess data concerning economic factors such as: Inflation rate, Unemployment rate and GDP per capita. We have chosen this three factors because we think they are the most important and a lot of other factors are highly correlated to this three. With this and social factors data sets we are trying to answer the question: "Are there typical characteristics of countries with high/low crime rates?".

In [ ]:
df8 = pd.read_csv('../input/economdata/economfaktoren.csv', encoding='latin1')
df8.head()

As we can see this data set goes from 2000 to 2015 and there are still some non-existent values that we need to fill with right sources or best techiniques possible. First we are going to change names of some columns and drop one column and then fill the data set.

In [ ]:
cols2 = list(df8.columns)
cols2[0] = 'Region'
cols2[2] = 'Year'
cols2[3] = 'Inflation rate(%)'
cols2[4] = 'Unemployment rate(%)'
cols2[5] = 'GDP per capita'
df8.columns = cols2
df8 = df8.drop(["Code"],1)
df8.head()

## FILLING THE DATA SET

In [ ]:
#filling the data set
df8.ix[0:4,2]=[4.2,4.9,5.1,11.7,11.2] #data on Afghanistan Inflation rate 2000-2004(sources: The World Bank, International Monetery Fund) )
df8.ix[0,4]=[176]#Afghanistan GDP per capita 2000(source:countryeconomy.com)
df8.set_index(['Region','Year'],inplace=True)#setting the index
df8.loc [['Argentina'], ['Inflation rate(%)']] = [-0.9,-1.1,25.9,13.4,4.4,9.6,10.9,8.8,8.6,6.3,10.5,9.8,10.0,10.6,38.0,26.7]#data on Argentinian inflation rate 2000-2015(sources:Index mundi, FocusEconomics)
df8.loc [['Antigua and Barbuda'], ['Unemployment rate(%)']] = [8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4,8.4]#there is no data on Antigua and Barbada unemployment rate by year so I just wrote an average unempoyment rate found on Index mundi 
df8.loc [['Bosnia and Herzegovina'],['Inflation rate(%)']] = [5,4.6,0.3,0.5,0.3,3.6,6.1,1.5,7.4,-0.4,2.1,3.7,2.1,-0.1,-0.9,-1]#data on BIH inflation rate 2000-2015(sources:Index mundi)
df8.loc [['Comoros'], ['Inflation rate(%)']] = [5.9,5.6,3.5,3.8,4.5,3,3.4,4.5,1.7,4.4,3.4,1.8,6.3,-4.3,1.35,2]#sources:KNOEMA.com, STATISTA.com
df8.loc [['Cuba'],['Inflation rate(%)']] = [2.6,4.9,7.1,4.1,3.1,7,5,3.1,3.4,-0.5,2.9,4.7,5.5,6,5.3,4.6]#source:Index mundi
df8.loc [['Iraq'],['GDP per capita']] = [1097,781,761,609,1391.82,1849.6,2351.81,3129.22,4521.03,3735.14,4502.75,5854.61,6651.12,6925.22,6703.07,4974.03]#source:countryeconomy.com

#Data on Dominica Unemployment rate
Youth_unemployment = [13.6,14,13.6,14.4,13.9,12.9,10.5,11.8,10.2,12,10.3,13.3,14.3,16.9,12.9,13.6]#data on youth employment in Dominica
Ratio = [23/13.6]*16 #Ratio between only year that I found on unemployment rate(2000,source:Index mundi) and youth unemployment rate in 2000 
New_unemployment_rate =[] 
end_index = len(Youth_unemployment)

for i in range(end_index):
    New_unemployment_rate.append(Youth_unemployment[i]*Ratio[i])
    
New_unemployment_rate = [round(x,1) for x in New_unemployment_rate] #rounding on one decimal place
df8.loc [['Dominica'],['Unemployment rate(%)']] = New_unemployment_rate 

##there wasn't much data on Grenada unemployment rate
df8.loc [['Grenada'],['Unemployment rate(%)']] = [12.5,9.9,14.4,14.4,14.4,18.8,22,22,25,25,25,25,28.8,32.5,29.3,28.9]#sources:Index mundi, GOV.gd,2002-2004(took average of 2001 and 2005),2006,2007(took average with respect to 2005 and 2008),2009,2010(avg of 2008,2011),2012(avg of 2011 and 2013)

df8.loc [['Guinea'],['Inflation rate(%)']] = [6.8,5.4,3,11,17.5,31.4,34.7,22.8,18.4,4.7,15.5,21.4,15.2,11.9,9.7,8.2]#source:Index mundi
df8.loc [['Lebanon'],['Inflation rate(%)']] = [-0.4,-0.4,1.8,1.3,1.7,-0.7,5.6,4.1,10.8,1.2,4,5,6.6,4.8,1.9,-3.7]#source:Index mundi
df8.loc [['Liberia'],['Inflation rate(%)']] = [5,8,14.2,10.3,7.8,10.8,7.3,11.4,17.5,7.4,7.3,8.5,6.8,7.6,9.9,7.7]#source:Index mundi
df8.loc [['Libya'],['Inflation rate(%)']] = [-2.9,-8.8,-9.8,-2.2,-2.2,2.7,1.5,6.3,10.4,2.5,2.8,15.5,6.1,2.6,2.4,9.8]#source:Statistic.com
df8.loc [['Maldives'],['Inflation rate(%)']] = [-1.2,0.7,4.2,-1.3,-1.7,1.3,2.7,6.8,12,4.5,6.1,11.3,10.9,3.8,2.1,1]#source:FactFish
df8.loc [['Montenegro'],['Inflation rate(%)']] = [29.9,23.7,19.7,7.5,3.1,3.4,2.9,4.3,8.8,3.5,0.7,3.5,4.1,2.2,-0.7,1.5]#source:KNOEMA.com
df8.loc [['Mozambique'],['Inflation rate(%)']] = [12.7,9.1,16.8,13.5,12.6,6.4,13.2,8.2,14.5,3.8,12.4,11.2,2.6,4.3,2.6,3.6]#source:KNOEMA.com
df8.loc [['Namibia'],['Inflation rate(%)']] = [10.2,10.2,12.7,7.1,4.1,2.3,5.0,6.5,9.1,9.5,4.9,5.0,6.7,5.6,5.3,3.4]#source:KNOEMA.com
df8.loc [['Oman'],['Inflation rate(%)']] = [-1.2,-0.8,-0.3,0.2,0.8,1.9,3.2,6.0,12.1,3.9,3.2,4.0,2.9,1.0,1.0,0.1]#source:KNOEMA.com
df8.loc [['Puerto Rico'],['Inflation rate(%)']] = [5.7,0.6,5,6.5,2.5,5.6,5.2,4.2,5.2,0.3,2.5,3.6,1.3,0.9,0.9,-0.3]#sources:FactFish,Index mundi,FocusEconomics
df8.loc [['Sao Tome and Principe'],['GDP per capita']] = [568,505.79,551.20,643.95,687.55,804.13,836.80,886.67,1126.46,1099.57,1129.76,1304.33,1380.95,1619.53,1824.38,1613.48]#source:IMF
df8.loc [['Seychelles'],['Unemployment rate(%)']] = [3.1,4.4,4.1,3.2,3.5,3.6,2.5,1.9,1.7,5.1,4.6,4.1,3.7,3.3,3.0,2.7]#sources:STATISTA, Index mundi
df8.loc [['Sierra Leone'],['Inflation rate(%)']] = [-0.9,2.6,0.1,4.0,12.9,13.7,10.5,11.6,8.2,7.5,7.2,6.8,6.6,5.5,4.6,6.7]#sources:KNOEMA.com
df8.loc [['Syria'],['Inflation rate(%)']] = [-3.8,3.0,-0.1,5.8,4.4,7.2,10.0,3.9,15.7,2.9,4.4,4.8,36.7,89.6,29.2,38.1]#sources:CIA World Factbook
df8.loc [['Syria'],['GDP per capita']] = [1177.63,1258.42,1263.01,1253.39,1408.85,1577.46,1762.25,2058.04,2557.2,2557.3,2806.7,2573,3608,2800,3300,2900]#sources:CIA, countryeconomy.com
df8.loc [['Tajikistan'],['Inflation rate(%)']] = [32.9,38.6,12.2,16.3,7.1,7.1,10.0,13.1,20.5,6.4,6.4,12.4,5.8,5.0,6.1,5.7]#source:CIA World Factbook
df8.loc [['Turkmenistan'],['Inflation rate(%)']] = [8.0,11.6,8.8,5.6,5.9,10.7,8.2,6.3,14.5,-2.7,4.4,5.3,5.3,6.8,6.0,7.4]#source:KNOEMA.com
df8.loc [['United Arab Emirates'],['Inflation rate(%)']] = [1.3,2.8,2.9,3.1,5.0,6.2,9.3,11.1,12.3,1.6,0.9,0.9,0.7,1.1,2.3,4.1]#source:KNOEMA.com
df8.loc [['Uzbekistan'],['Inflation rate(%)']] = [25,27.3,27.3,12.5,7.3,10.7,13.1,11.2,13.1,12.3,12.3,12.4,11.9,11.7,9.1,8.5]#source:KNOEMA.com
df8.loc [['Venezuela'],['Inflation rate(%)']] = [13,12.3,31.2,31.1,22.4,16.0,15.8,18.7,30.4,27.1,28.2,26.1,21.1,40.6,62.2,121.7]#source:Index mundi
df8.loc [['Venezuela'],['GDP per capita']] = [4783.53,4926.31,3655.98,3232.52,4271.37,5432.69,6735.80,8318.80,11227.23,11536.15,13545.26,10741.58,12755.00,12237.19,15692.41,30775.00]#source:CEIC
df8.loc [['Yemen'],['Inflation rate(%)']] = [4.6,11.9,12.2,10.8,12.5,11.8,10.8,7.9,19.0,5.4,11.2,19.5,9.9,11.0,8.1,12]#source:statistic.com
df8.loc [['Zimbabwe'],['Inflation rate(%)']] = [4.5,-37.2,-34.4,-8.6,113.6,-31.5,33.0,-72.7,157.0,6.2,3.0,3.5,3.7,1.6,-0.2,-2.4]#source:KNOEMA.com


df8.head()

This procedure wasn't the easiest because there a lot of countries which aren't really transparent or they just don't keep track on some of the economic factors. We mostly tried to find another sources other than our primary source (THEGLOBALECONOMY.COM). These sources included sites: Index mundi, FOCUSECONOMICS.com, STATISTA, CIA World Factbook, GOV.gd, FACTFISH.com, KNOEMA.com, Interational Monetary Fund, COUNTRYECONOMY.com, STATISTIC.com. We couldn't find a lot of information about Dominica and Grenada so we used some techinques. For Dominica we found only one year (2000) concerning unemployment rate but we managed to find another data set that should be highly correlated to Dominica unemployment rate and that is youth unemployment rate. We took ratio between unemployment rate (2000) and youth unemployment rate (2000) and multiplied youth unenmployment rate list with that ratio. In the case of Grenada unemployment rate set, we had a lot of holes in our list, so what we did was mostly manual interpolation with a help of some internet sites (mostly KNOEMA.com) that provided average unemployment rates for some years. 

Preparing data set on economic factors for merging

In [ ]:
#setting all values as floats
df8.loc[:,'Inflation rate(%)'] = df8.loc[:,'Inflation rate(%)'].astype('float')/100
#we want to have unemployment per capita for our last model so I am dividing whole column with 100
df8.loc[:,'Unemployment rate(%)'] = df8.loc[:,'Unemployment rate(%)'].astype('float')/100
df8.loc[:,'GDP per capita'] = df8.loc[:,'GDP per capita'].astype('float') 

cols3 = list(df8.columns)
cols3[0] = 'Inflation rate' 
cols3[1] = 'Unemployment rate per capita'
df8.columns = cols3
#pd.set_option('display.max_rows',3000)
df_econ_max=df8
df8.head()

## PLOTTING

### GDP PER CAPITA PLOT

In [ ]:
df8_plot=df8.reset_index()

column_name='GDP per capita'
data = [ dict(
        type = 'choropleth',
        locations = df8_plot['Region'].astype(str),
        locationmode = 'country names',
        z = df8_plot[column_name].astype(float),
        #text = df8['Code'].astype(str),
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = column_name),
      ) ]

layout = dict(
    title = 'world plot of GDP per capita (2000)',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d8-world-map' )


### INFLATION RATE PLOT

In [ ]:
column_name='Inflation rate'
data = [ dict(
        type = 'choropleth',
        locations = df8_plot['Region'].astype(str),
        locationmode = 'country names',
        z = df8_plot[column_name].astype(float),
        #text = df8['Code'].astype(str),
        colorscale = [[0,"rgb(172, 10, 5)"],[0.35,"rgb(190, 60, 40)"],[0.5,"rgb(245, 100, 70)"],\
            [0.6,"rgb(245, 120, 90)"],[0.7,"rgb(247, 137, 106)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = column_name),
      ) ]

layout = dict(
    title = 'world plot of Inflation rate (2000)',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d8-world-map' )

### UNEMPLOYMENT RATE PER CAPITA PLOT

In [ ]:
column_name='Unemployment rate per capita'
data = [ dict(
        type = 'choropleth',
        locations = df8_plot['Region'].astype(str),
        locationmode = 'country names',
        z = df8_plot[column_name].astype(float),
        #text = df8['Code'].astype(str),
        colorscale = [[0,"rgb(5, 172, 10)"],[0.35,"rgb(40, 190, 60)"],[0.5,"rgb(70, 245, 100)"],\
            [0.6,"rgb(90, 245, 120)"],[0.7,"rgb(106, 247, 137)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = column_name),
      ) ]

layout = dict(
    title = 'world plot of Unemployment rate per capita (2000)',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d8-world-map' )


# **SOCIAL FACTORS**

The following datasets are observations for 1950-2015 for 201-232 different countries.

**Total population**

In [ ]:
df_total_pop = pd.read_csv('../input/population-un-data/Total_population_both_sexes.csv',
                           sep=';', encoding='latin', decimal = ',',skiprows=[0])
df_total_pop.head()

We reshape the datasets into tidy format and do some preprocessing...

In [ ]:
df_total_pop_formatted = pd.melt(df_total_pop, ['Country'], var_name = 'Year', value_name = 'Total population in 1,000')
df_total_pop_formatted.columns = ['Region', 'Year', 'Total population in 1,000']
df_total_pop_formatted = df_total_pop_formatted.sort_values(by = ['Region','Year'])
country_names = df_total_pop_formatted.Region
df_total_pop_formatted.Year = df_total_pop_formatted.Year.astype(int)
df_total_pop_formatted.iloc[:,2] = df_total_pop_formatted.iloc[:,2].str.replace(' ', '')
df_total_pop_formatted.iloc[:,2] = df_total_pop_formatted.iloc[:,2].astype(np.float64)
df_total_pop_formatted = df_total_pop_formatted.loc[df_total_pop_formatted['Year'] >1999,:]

In [ ]:
country_names2 = country_names.drop_duplicates()

There are some names we want to change...

In [ ]:
false_names = ['Ivory Coast+A168', 'Viet Nam','Cabo Verde']
new_names = ['Ivory Coast', 'Vietnam','Cape Verde']
df_total_pop_formatted = df_total_pop_formatted.replace(to_replace=false_names, value=new_names)
df_total_pop_formatted = df_total_pop_formatted.sort_values(by = ['Region','Year'])

In [ ]:
country_names = df_total_pop_formatted.Region
country_names2 = country_names.drop_duplicates()

**Population density**

In [ ]:
df_pop_density = pd.read_csv('../input/population-un-data/Population_density.csv',sep=';',encoding='latin',decimal = ',',skiprows = [0])
df_pop_density_formatted = pd.melt(df_pop_density, ['Country'], var_name = 'Year', value_name = 'Population density (persons per km2)')
df_pop_density_formatted.columns = ['Region', 'Year','Population density (persons per km2)']
df_pop_density_formatted = df_pop_density_formatted.sort_values(by = ['Region','Year'])
df_pop_density_formatted.Year = df_pop_density_formatted.Year.astype(int) 
df_pop_density_formatted = df_pop_density_formatted.loc[df_pop_density_formatted['Year'] >1999,:]

In [ ]:
country_names3 = df_pop_density_formatted.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Bolivia','Cape Verde','Hong Kong', 'Macao', 'Taiwan', 'Curacao','Czech Republic', 'Ivory Coast', 'Korea', 'Falkland Islands','Holy See',
             'Iran', 'Laos', 'Micronesia','Moldova','Russia','Sint Maarten', 'Syria', 'Macedonia','Tanzania', 'Venezuela','Vietnam']
df_pop_density_formatted = df_pop_density_formatted.replace(to_replace=false_names, value=new_names)
df_pop_density_formatted = df_pop_density_formatted.sort_values(by = ['Region','Year'])

country_names2 = country_names2.set_value(country_names2.size, 'Holy See')

**Crude birth rate**

In [ ]:
df_crude_birth_rate = pd.read_csv('../input/population-un-data/Crude_birth_rate.csv',sep=';',encoding='latin',decimal = ',',skiprows = [0])
df_crude_birth_rate.head()

We split the time interval columns into 5 different ones by dividing the value of each column by 5. 

In [ ]:
cols = df_crude_birth_rate.columns
col_loc = 0
for year in cols:
    if 'Country' in year:
        colloc=+1
        continue
    else:
        curr_col = df_crude_birth_rate[year]
        new_col = curr_col/5
        first, last = year.split('-')
        
        for i in range(int(first),int(last)):
            df_crude_birth_rate[str(i)] = new_col
        
        df_crude_birth_rate = df_crude_birth_rate.drop(year,1)
        
        col_loc=+1
df_crude_birth_rate[str(i+1)] = new_col

We reshape again into tidy format.

In [ ]:
df_crude_birth_rate_formatted = pd.melt(df_crude_birth_rate, ['Country'], var_name = 'Year', value_name = 'Crude birth rate (births per 1,000)')
df_crude_birth_rate_formatted = df_crude_birth_rate_formatted.sort_values(by = ['Country','Year'])
df_crude_birth_rate_formatted.columns = ['Region', 'Year', 'Crude birth rate (births per 1,000)']
df_crude_birth_rate_formatted.Year = df_crude_birth_rate_formatted.Year.astype(int) 
df_crude_birth_rate_formatted = df_crude_birth_rate_formatted.loc[df_crude_birth_rate_formatted['Year'] >1999,:]

In [ ]:
country_names3 = df_crude_birth_rate_formatted.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Bolivia','Cape Verde','Hong Kong', 'Macao', 'Taiwan', 'Curacao', 'Czech Republic','Ivory Coast', 'Korea',
             'Iran', 'Laos', 'Micronesia','Moldova','Russia','Syria', 'Macedonia','Tanzania', 'Venezuela','Vietnam']
df_crude_birth_rate_formatted = df_crude_birth_rate_formatted.replace(to_replace=false_names, value=new_names)
df_crude_birth_rate_formatted = df_crude_birth_rate_formatted.sort_values(by = ['Region','Year'])

**Total fertility rate**

In [ ]:
df_fertility = pd.read_csv('../input/population-un-data/Total_fertility_rate.csv', sep = ',',delimiter='"')
df_fertility.head()

Seems like 'pandas.read_csv' does not like the double quotes. I tried several optional arguments, nothing worked fine. I decided to just delete the double quotes from the csv file and then read the csv again.

In [ ]:
#!tr -d '"' <../input/population-un-data/Total_fertility_rate.csv >Total_fertility_rate1.csv

df_fertility = pd.read_csv('../input/population-un-data/Total_fertility_rate1.csv', sep=',', usecols = (0,1,3))
df_fertility.columns = ['Country', 'Year', 'Total fertility rate']

We do not need any predicted values for the future and remove them.

In [ ]:
df_fertility2 = df_fertility.loc[17:29,:]

for k in range(1,240):
    df_fertility2 = df_fertility2.append(df_fertility.loc[k*30 + 17:k*30 + 29,:])

Similar to the crude birht rate we extend the time interval to 5 different rows.

In [ ]:
df_fertility3 = pd.DataFrame(columns = ['Country', 'Year', 'Total fertility rate'])
for index, row in df_fertility2.iterrows():
    region = row['Country']
    year = row['Year']
    val = row['Total fertility rate']
    
    first, last = year.split('-')
    for i in range(int(first),int(last)):
        new_row = pd.DataFrame([[region,i,val]], columns = ['Country', 'Year', 'Total fertility rate'])
        df_fertility3 = pd.concat([df_fertility3,new_row])
df_fertility3 = df_fertility3.sort_values(by = ['Country', 'Year'])
df_fertility3.columns = ['Region', 'Year', 'Total fertility rate']
df_fertility3.Year = df_fertility3.Year.astype(int)
df_fertility3 = df_fertility3.loc[df_fertility3['Year'] >1999,:]

In [ ]:
country_names3 = df_fertility3.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Bolivia','Cape Verde','Curacao','Czech Republic','Korea', 'Falkland Islands',
             'Iran', 'Laos', 'Polynesia','Moldova','Russia','Sint Maarten', 'Syria', 'Macedonia','Tanzania', 'Venezuela','Vietnam']
df_fertility3 = df_fertility3.replace(to_replace=false_names, value=new_names)
df_fertility3 = df_fertility3.sort_values(by = ['Region','Year'])

country_names2 = country_names2.set_value(country_names2.size,'Polynesia')

**Enrolment in first grade of primary education**

In [ ]:
#!tr -d '"' <../input/population-un-data/Enrolment_in_Grade_1_of_primary_education.csv >Enrolment_in_Grade_1_of_primary_education1.csv

df_education_grade1 = pd.read_csv('../input/population-un-data/Enrolment_in_Grade_1_of_primary_education1.csv', sep = ",", usecols = (0,1,2,5))
df_education_grade1.columns = ['Region', 'Year', 'Sex', 'Enrolment in Grade 1 of primary education']
df_education_grade1 = df_education_grade1.loc[df_education_grade1['Year'] >1999,:]
#df_education_grade1

In [ ]:
country_names3 = df_education_grade1.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Ivory Coast','Korea','Hong Kong', 'Laos', 'Lybia','Macao','Netherlands Antilles','Palestine','Russia', 'Syria', 'Macedonia',
             'Tanzania', 'Venezuela','Vietnam']
df_education_grade1 = df_education_grade1.replace(to_replace=false_names, value=new_names)
df_education_grade1 = df_education_grade1.sort_values(by = ['Region','Year'])

In [ ]:
country_names2 = country_names2.set_value(country_names2.size,'Netherlands Antilles')
country_names2 = country_names2.set_value(country_names2.size,'Palestine')
country_names2 = country_names2.set_value(country_names2.size,'Réunion')

In [ ]:
df_pop_ed = pd.merge(df_total_pop_formatted,df_education_grade1, on=['Region','Year'], how='inner')
pop = df_pop_ed.iloc[:,2]
ed = df_pop_ed.iloc[:,4]
ed_rate = ed/(pop*1000)*100
df_pop_ed['Enrolment in first grade of primary education (rate in percent)'] = ed_rate
df_education_grade1 = df_pop_ed.drop(['Total population in 1,000', 'Enrolment in Grade 1 of primary education'],axis=1)

In [ ]:
df_education_grade1_female = df_education_grade1.loc[df_education_grade1['Sex'] == 'Female',:]
df_education_grade1_female = df_education_grade1_female.drop(['Sex'], axis=1)
df_education_grade1_female.columns = ['Region', 'Year', 'Enrolment in first grade of primary education (rate in percent, females)']

df_education_grade1_all_genders = df_education_grade1.loc[df_education_grade1['Sex'] == 'All genders',:]
df_education_grade1_all_genders = df_education_grade1_all_genders.drop(['Sex'], axis=1)
df_education_grade1_all_genders.columns = ['Region', 'Year', 'Enrolment in first grade of primary education (rate in percent, all genders)']

**Primary education drop-out rate**

In [ ]:
#!tr -d '"' <../input/population-un-data/Primary_education_Drop-out_rate.csv >Primary_education_Drop-out_rate1.csv

df_education_drop_out = pd.read_csv('../input/population-un-data/Primary_education_Drop-out_rate1.csv', usecols = (0,1,2,5))
df_education_drop_out.columns = ['Region','Year', 'Sex', 'Primary Education drop-out rate (in percent)']
df_education_drop_out.Year = df_education_drop_out.Year.astype(int)
df_education_drop_out.iloc[:,3] = df_education_drop_out.iloc[:,3].astype(np.float64)
df_education_drop_out = df_education_drop_out.loc[df_education_drop_out['Year'] >1999,:]

In [ ]:
country_names3 = df_education_drop_out.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Ivory Coast','Hong Kong','Laos','Macao','Palestine','Russia', 'Syria', 'Macedonia',
             'Tanzania', 'Venezuela','Vietnam']
df_education_drop_out = df_education_drop_out.replace(to_replace=false_names, value=new_names)
df_education_drop_out = df_education_drop_out.sort_values(by = ['Region','Year'])

In [ ]:
df_education_drop_out_female = df_education_drop_out.loc[df_education_drop_out['Sex'] == 'Female',:]
df_education_drop_out_female = df_education_drop_out_female.drop(['Sex'], axis=1)
df_education_drop_out_female.columns = ['Region','Year','Primary Education drop-out rate (females)']

df_education_drop_out_male = df_education_drop_out.loc[df_education_drop_out['Sex'] == 'Male',:]
df_education_drop_out_male = df_education_drop_out_male.drop(['Sex'], axis=1)
df_education_drop_out_male.columns = ['Region','Year','Primary Education drop-out rate (in percent, males)']

df_education_drop_out_all_genders = df_education_drop_out.loc[df_education_drop_out['Sex'] == 'All genders',:]
df_education_drop_out_all_genders = df_education_drop_out_all_genders.drop(['Sex'], axis=1)
df_education_drop_out_all_genders.columns = ['Region','Year','Primary Education drop-out rate (in percent, all genders)']

**Graduates from tertiary education**

In [ ]:
#!tr -d '"' <../input/population-un-data/Graduates_from_tertiary_education_both_sexes.csv >Graduates_from_tertiary_education_both_sexes1.csv

df_education_graduates = pd.read_csv('../input/population-un-data/Graduates_from_tertiary_education_both_sexes1.csv',usecols = (0,1,2,5))
df_education_graduates.columns = ['Region', 'Year', 'Sex','Graduates from tertiary education']
df_education_graduates = df_education_graduates.loc[df_education_graduates['Year'] >1999,:]

In [ ]:
country_names3 = df_education_graduates.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Hong Kong', 'Laos','Macao','Palestine','Russia', 'Syria','Macedonia','United Kingdom','Tanzania', 'Venezuela','Vietnam']
df_education_graduates = df_education_graduates.replace(to_replace=false_names, value=new_names)
df_education_graduates = df_education_graduates.sort_values(by = ['Region','Year'])

In [ ]:
df_pop_ed1 = pd.merge(df_total_pop_formatted,df_education_graduates, on=['Region','Year'], how='inner')
pop = df_pop_ed1.iloc[:,2]
ed = df_pop_ed1.iloc[:,4]
ed_rate = ed/(pop*1000)*100
df_pop_ed1['Graduates from tertiary education (rate in percent)'] = ed_rate
df_education_graduates = df_pop_ed1.drop(['Total population in 1,000', 'Graduates from tertiary education'],axis=1)

In [ ]:
df_education_graduates_female = df_education_graduates.loc[df_education_graduates['Sex'] == 'Female',:]
df_education_graduates_female = df_education_graduates_female.drop(['Sex'], axis=1)
df_education_graduates_female.columns = ['Region','Year','Graduates from tertiary education (rate in percent, females)']

df_education_graduates_all_genders = df_education_graduates.loc[df_education_graduates['Sex'] == 'All genders',:]
df_education_graduates_all_genders = df_education_graduates_all_genders.drop(['Sex'], axis=1)
df_education_graduates_all_genders.columns = ['Region','Year','Graduates from tertiary education (rate in percent, all genders)']

**Female to male ratio students at tertiary level education**

In [ ]:
df_edu = pd.read_csv('../input/population-un-data/education_theglobaleconomy.csv',usecols=(0,2,3),na_values=' ')
df_edu.rename(columns = {'Country':'Region',
                            ' Female to male ratio students at tertiary level education':
                             'Female to male ratio students at tertiary level education'},inplace=True)
df_edu = df_edu.loc[df_edu['Year'] <2016,:]

In [ ]:
country_names3 = df_edu.reset_index().Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Brunai Darussalam','Myanmar','North Korea','Congo','Korea']
df_edu = df_edu.replace(to_replace=false_names, value=new_names)
df_edu = df_edu.sort_values(by = ['Region','Year'])

We intersect the country names from all dataframes

In [ ]:
#country_names2
names_dens = df_pop_density_formatted.Region.drop_duplicates()
names_dens = names_dens.tolist()

names_birth = df_crude_birth_rate_formatted.Region.drop_duplicates()
names_birth = names_birth.tolist()

names_fert = df_fertility3.Region.drop_duplicates()
names_fert = names_fert.tolist()

names_ed1 = df_education_grade1_all_genders.Region.drop_duplicates()
names_ed1 = names_ed1.tolist()

names_ed_drop = df_education_drop_out_all_genders.Region.drop_duplicates()
names_ed_drop = names_ed_drop.tolist()

names_edu = df_edu.Region.drop_duplicates()
names_edu = names_edu.tolist()

names1 = [itm for itm in country_names2 if itm in names_dens]
names2 = [itm for itm in names_birth if itm in names_fert]
names3 = [itm for itm in names_ed1 if itm in names_ed_drop]
names4 = [itm for itm in names1 if itm in names2]
names5 = [itm for itm in names3 if itm in names4]
names = [itm for itm in names5 if itm in names_edu]

In [ ]:
df_pop = df_total_pop_formatted[df_total_pop_formatted.Region.isin(names)==True]
df_dens = df_pop_density_formatted[df_pop_density_formatted.Region.isin(names)==True]
df_birth = df_crude_birth_rate_formatted[df_crude_birth_rate_formatted.Region.isin(names)==True]
df_fert = df_fertility3[df_fertility3.Region.isin(names)==True]
df_ed1 = df_education_grade1_all_genders[df_education_grade1_all_genders.Region.isin(names)==True]
df_drop_out = df_education_drop_out_all_genders[df_education_drop_out_all_genders.Region.isin(names)==True]
#df_grad = df_education_graduates_all_genders[df_education_graduates_all_genders.Region.isin(names)==True]
df_edu2 = df_edu[df_edu.Region.isin(names)==True]

In [ ]:
df_social = pd.merge(df_pop, df_dens, on = ['Region','Year'], how = 'outer')
df_social = pd.merge(df_social, df_birth, on = ['Region','Year'], how = 'outer')
df_social = pd.merge(df_social, df_fert, on = ['Region','Year'], how = 'outer')
df_social = pd.merge(df_social, df_ed1, on = ['Region','Year'], how = 'outer')
df_social = pd.merge(df_social, df_drop_out, on = ['Region','Year'], how = 'outer')
#df_social = pd.merge(df_social, df_grad, on = ['Region','Year'], how = 'outer')
df_social = pd.merge(df_social, df_edu2, on = ['Region','Year'], how = 'outer')

Handling missing values with fancyimpute

In [ ]:
from fancyimpute import IterativeImputer

imp_cols = df_social.iloc[:,2:9].columns.values
to_impute = df_social.iloc[:,2:9]

df_social.iloc[:,2:9] = pd.DataFrame(IterativeImputer().fit_transform(to_impute),columns = imp_cols)
df_social = df_social.sort_values(by = ['Region','Year'])
df_social = df_social.set_index(['Region','Year'])

In [ ]:
df_social.isna().sum().sum()

**Some more preprocessing and joining the 3 Big data frames--- Social, Economic and Crime Factors**

We check the countries' names of the other dataframes ...

In [ ]:
country_names3 = df8.reset_index().Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Brunai Darussalam','Myanmar','Congo','Korea','United States of America']
df8 = df8.reset_index()
df8.Region = df8.Region.replace(to_replace=false_names, value=new_names)
df8 = df8.sort_values(by = ['Region','Year'])
econom_isi = df8[df8.Region.isin(names)==True]
df8 = df8.set_index(['Region','Year'])
df8.head()

# Merging

In [ ]:
unodc_mult = unodc_mult.reset_index()
country_names3 = unodc_mult.Region
country_names3 = country_names3.drop_duplicates()
false_names = country_names3[country_names3.isin(country_names2)==False]
false_names = false_names.tolist()
false_names

In [ ]:
new_names = ['Bahamas','Brunai Darussalam','Myanmar', 'Cape Verde','Congo democratic','Congo', 'Ivory Coast','Falkland Islands',
             'Faeroe Islands','Gambia','Guernsey','Hong Kong','Iran','Iraq','Jersey','North Korea','Korea','Kosovo',
             'Kosovo under UNSCR 1244','Macao','Macau','Micronesia','Sint Maarten',
            'Saint Kitts and Nevis','Saint Vincent and the Grenadines','Syria','Macedonia',
            'United Kingdom (Northern Ireland)','United Kingdom (Scotland)',
            'United States of America','British Virgin Islands','West Bank']
#unodc_mult = unodc_mult.reset_index()
unodc_mult.Region = unodc_mult.Region.replace(to_replace = false_names, value = new_names)
unodc_mult = unodc_mult.sort_values(by = ['Region','Year'])
unodc_mult_isi = unodc_mult[unodc_mult.Region.isin(names)==True]
unodc_mult_isi

unodc_mult = unodc_mult.set_index(['Region','Year'])

In [ ]:
df_social = df_social.reset_index()
unodc_mult_isi = unodc_mult_isi.reset_index()
econom_isi = econom_isi.reset_index()

names_social = df_social.Region.drop_duplicates().tolist()
names_uno = unodc_mult_isi.Region.drop_duplicates().tolist()
names_econom = econom_isi.Region.drop_duplicates().tolist()

names = [itm for itm in names_social if itm in names_econom]
names = [itm for itm in names if itm in names_uno]

social_isi = df_social[df_social.Region.isin(names)==True]
unodc_mult_isi = unodc_mult_isi[unodc_mult_isi.Region.isin(names)==True]
econom_isi = econom_isi[econom_isi.Region.isin(names)==True]
econom_isi = econom_isi.drop(['index'],axis=1)

df_all = pd.merge(unodc_mult_isi,econom_isi, on=['Region','Year'],how='outer')
df_all = pd.merge(df_all,social_isi, on=['Region','Year'],how='outer')

df_social = df_social.set_index(['Region','Year'])
unodc_mult_isi = unodc_mult_isi.set_index(['Region','Year'])
econom_isi = econom_isi.set_index(['Region','Year'])

df_all = df_all.drop(['index'],axis=1)

df_all = df_all.sort_values(by = ['Region','Year'])

In [ ]:
df_all = df_all.drop(['Assault_Abs', 'Kidnapping_Abs', 'Theft_Abs', 'Robbery_Abs',
       'Burglary_Abs', 'Domestic_Burglary_Abs', 'Theft_of_Private_Cars_Abs',
       'Motor_Vehicle_Theft_Abs', 'Total_Sexual_Violence_Abs', 'Rape_Abs',
       'Sexual_Offences_ag_Children_Abs'], axis = 1)

In [ ]:
df_all.columns

In [ ]:
df_all.rename(columns = {'Assault_Rel':'Assault rate per 100,000 population',
                        'Kidnapping_Rel':'Kidnapping rate per 100,000 population',
                        'Theft_Rel':'Theft rate per 100,000 population',
                        'Robbery_Rel':'Robbery rate per 100,000 population',
                        'Burglary_Rel':'Burglary rate per 100,000 population',
                        'Domestic_Burglary_Rel':'Domestic Burglary rate per 100,000 population',
                        'Theft_of_Private_Cars_Rel':'Theft of Private Cars rate per 100,000 population',
                        'Motor_Vehicle_Theft_Rel':'Motor Vehicle Theft rate per 100,000 population',
                        'Total_Sexual_Violence_Rel':'Total Sexual Violence rate per 100,000 population',
                        'Rape_Rel':'Rape rate per 100,000 population',
                        'Sexual_Offences_ag_Children_Rel':'Sexual Offences agains Children rate per 100,000 population'},
                inplace = True)

In [ ]:
df_all_red = df_all.loc[df_all['Year'] > 2002,:]
df_all_red = df_all_red.loc[df_all_red['Year'] < 2014,:]

In [ ]:
print('Those are the countries that are not suitable for interpolation')
c = list()
for country in names:
    curr_country = df_all_red.loc[df_all_red['Region']==country,:]
    first = curr_country.iloc[2,:]
    last = curr_country.iloc[10,:]
    
    if first.isna().sum() > 0:
        print(country)
        c.append(country)
        continue
    if last.isna().sum()  > 0:
        print(country)
        c.append(country)
len(c)

Unfortunately these are too many...

In [ ]:
df_social_max=df_social

<a id="6"></a> <br>
# 3. Are there typical characteristics of countries with high/low crime rates?

In [ ]:
my_countries=unodc_mult.index.levels[0]
plot_countries=["United States","Sweden","Austria","Mexico","Croatia"]
fields_rel=[f+"_Rel" for f in fields]
unodc_avg_region_all=pd.DataFrame()
unodc_avg_region=unodc_mult.loc[my_countries][fields_rel]
unodc_avg_region-=unodc_avg_region.mean()
unodc_avg_region/=unodc_avg_region.std()
unodc_avg_region_all["crime_score_raw"]=unodc_avg_region.mean(axis=1)

unodc_avg_region_all["corruption_score"]=0
for cntr in unodc_avg_region_all.index.levels[0]:
    try:
        unodc_avg_region_all["corruption_score"].loc[cntr]=corruption_avg["corruption_score"].loc[cntr]
    except Exception:
        continue
unodc_avg_region_all["crime_score"]=unodc_avg_region_all["crime_score_raw"]+unodc_avg_region_all["corruption_score"]/4
#unodc_avg_region_all["crime_score"]-=unodc_avg_region_all["crime_score"].min()
unodc_avg_region_all[:20]

In [ ]:
unodc_avg_region_all.drop(columns=["corruption_score","crime_score_raw"],inplace=True)

In [ ]:
import matplotlib.pyplot as plt
unodc_avg_region_all.loc[plot_countries].unstack(level=0).plot(figsize=(16, 9))

In [ ]:
import numpy as np
models={}
unodc_avg_region_all["crime_score_lin"]=0
crime_score_grad=pd.Series()
for c in unodc_avg_region_all.index.levels[0]:
    tmp=unodc_avg_region_all.loc[c]
    tmp=tmp[~tmp["crime_score"].isna()]
    if tmp.shape[0]<12:
        continue
    alpha=np.polyfit(
        (tmp.index-2000).tolist(), 
        tmp["crime_score"].tolist(), 
        2
    )
    for y in range(3,15):
        unodc_avg_region_all["crime_score_lin"].loc[c,2000+y]=y*y*alpha[0]+y*alpha[1]+alpha[2]
    crime_score_grad[c]=abs(alpha[0]) # taking the coefficient of x**2 as this is telling wether or not there was a change

In [ ]:
import matplotlib.pyplot as plt
unodc_avg_region_all.loc[plot_countries].unstack(level=0).plot(figsize=(16, 9))

In [ ]:
interesting_countries=crime_score_grad.sort_values(ascending=False).head(7).index

import matplotlib.pyplot as plt
unodc_avg_region_all["crime_score"].loc[interesting_countries].unstack(level=0).plot(figsize=(16, 9))

In [ ]:
trend_plot=unodc_avg_region_all[["crime_score"]].loc[interesting_countries].copy()
trend_plot=trend_plot.merge(df_social_max,left_index=True,right_index=True,how="outer")
trend_plot=trend_plot.merge(df_econ_max,left_index=True,right_index=True,how="outer")
trend_plot-=trend_plot.mean()
trend_plot/=trend_plot.std()
trend_plot.loc["Australia"]

In [ ]:
for c in interesting_countries:
    trend_plot[["crime_score","GDP per capita","Inflation rate","Unemployment rate per capita","Primary Education drop-out rate (in percent, all genders)"]].loc[c].plot(title=c,figsize=(16, 9))

WE have seen how crime indicators moves with other socio-economiv indicators. Let us quantify it in a corr plot.

Before we proceede we need to prepare the final dataframe a bit more. One thing I do want to note. There we a LOT! of NaN values, and MICE (ITerativeImputer) did not impute the values as expected (only on crime data since thats were we had a lot of NaN on socio and economic data it worked splendid). Interpolation (the logical choice) was out of the question since we did not have data points on both ends, i.e. there were missing from the gekko or entirely from the end. So we needed to compromise regarding interpolation, polynomial where possible and bfil, ffil otherwise.

To create a target variable we will use the normalised aggregated crime rates from before:

In [ ]:
crime_normalised=unodc_avg_region_all["crime_score"].reset_index()

In [ ]:
df_all_red["total_crime"]=crime_normalised["crime_score"]

In [ ]:
df_all_red.interpolate(method='linear',inplace=True)

In [ ]:
df_all_red.fillna(method="bfill",inplace=True)

In [ ]:
df_all_red.fillna(method="ffill",inplace=True)

In [ ]:
df_all_red.head()

In [ ]:

df_all_red.drop(['Region'],axis=1,inplace=True)

# calculate the correlation matrix
corr = df_all_red.corr()

# plot the heatmap
sns_plot2=sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)


(interesting and some) Conclusions from the correlation matrix: 

*  WE can see some correlation between crimes and inflation rate & GDP rate
*  Between crimes we can see (and confirm) that assault rate is correted with rape rate 
*  And socio-economic that GDP is correlated to Female to male student ratio

To answer the last question we are going to build a model and try to predict all of the crimes combined using other (socio-economic) factors

<a id="7"></a> <br>
# Question 5: Are there also country characteristics that predict trends in crime rates or types of crimes? 

We have joined all of the sets. Aggregated all of the crimes  into one column and now will use other variables to predict it (regression problem). If we were to use light gradient boosting method (lgbm) than there is also an option to use showcase important features, i.e. feature that had the most predictive characteristics. When we have such information than typical characteristics of countries with high/low crime rates are those features that had the most importance when predicting.

In [ ]:
df_all.head()

In [ ]:
y_pred=df_all_red["total_crime"]

In [ ]:
df_all_red.drop(['total_crime'], axis=1, inplace=True)

In [ ]:
X_train=df_all_red.loc[df_all_red['Year'] < 2013,:]

In [ ]:
X_test1=df_all_red.loc[df_all_red['Year'] > 2012,:]

In [ ]:
X_test1.head()

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

In [ ]:
def train_model(X=X_train, X_test=X_test1, y=y_pred, params=None, folds=folds, model_type='lgb', plot_feature_importance=False, model=None):

    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 50000, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='mae',verbose=10000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_tr.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_tr.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X_tr.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_tr.columns), ntree_limit=model.best_ntree_limit)
            
        if model_type == 'rcv':
            model = RidgeCV(alphas=(0.01, 0.1, 1.0, 10.0, 100.0, 1000.0), scoring='neg_mean_absolute_error', cv=5)
            model.fit(X_train, y_train)
            print(model.alpha_)

            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
            print('')
            
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
            print('')
            
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='MAE', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid, y_pred_valid))

        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction
    
    else:
        return oof, prediction

In [ ]:
lgb_params = {'num_leaves': 80,
              'min_child_weight': 28,
              'min_split_gain': 0.745,
          'min_data_in_leaf': 79,
          'objective': 'huber',
          'max_depth': 25,
          'learning_rate': 0.01,
          "boosting": "gbdt",
          "bagging_freq": 4,
          "bagging_fraction": 0.8126672064208567,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1058,
          'reg_lambda': 0.2209,
          'feature_fraction': 0.9201
         }
oof_lgb, prediction_lgb, feature_importance = train_model(params=lgb_params, model_type='lgb', plot_feature_importance=True)

Usually I extend this code and apply the Bayesian Hyperparameter optimisation but in this case we are not that interested in having the best of the best predictions and it would take away from memory and run-time and not help that much with the predictions, so I ommited it and used parameters from another project (they are all anyways in a standard intervall for a regression).

Interpretation: We need to neglect the cirminal variables (since I did not filter them out) and just look at the socio-economic. We can see that population, fertility, Unemployment and education are deal breakers!